In [1]:
# Load scripts without installing
import sys, os.path
sys.path.insert(0, os.path.abspath(".."))

# Inconsistency with Near Repeat Calculator

Currently, the most commonly-used implementation of the Knox test - at least within the criminological literature - is the [Near Repeat Calculator (NRC)](http://www.cla.temple.edu/center-for-security-and-crime-science/projects/#near-repeat-calculator), which is a freely-available standalone tool developed by Jerry Ratcliffe. The tool was developed for research purposes, and allows users to apply a version of the Knox test - the same as that which is implemented here - to any set of crime events provided in a specified format. The [manual (PDF)](https://liberalarts.temple.edu/sites/liberalarts/files/NearRepeatsManual.pdf) for the tool provides details of its operation and the nature of the algorithm implemented.

It appears that the results given by the NRC do not match those provided by the code presented here. This Notebook demonstrates the inconsistency and outlines a possible explanation.

## Application to test data

The inconsistency is demonstrated via application of the algorithms to the test dataset `chicago_burglary_2014_2017.csv`, pre-processed as explained in the [accompanying notebook](Prepare%20example%20data.ipynb). The data is already in the required format for the NRC. For this analysis, we'll work with data from 2016 only.

The settings used when applying the NRC - chosen for no particular reason - are as follows:

- **Spatial bandwidth**: 100 metres
- **Number of spatial bands**: 3
- **Temporal bandwidth**: 2 days
- **Number of temporal bands**: 2
- **Significance level (i.e. number of iterations)**: p=0.05 (i.e. 20)
- **Distance setting**: Euclidean

The NRC produces two output files (both accompanying this Notebook): a '[summary](nrc_output/chicago_burglary_2016_NRsummary.htm)' document in HTML format, containing headline results, and a '[verbose](nrc_output/chicago_burglary_2016_NRverbose.csv)' file in CSV format, which holds the results of intermediate calculations. The latter includes the *Observed frequencies distribution*, which is essentially the observed contingency table for the underlying data - this contains the number of incident pairs falling within each spatio-temporal band.

For the test data, the *Observed frequencies distribution* is as follows:

| &nbsp;        |  0 to 2 | 3 to 4 |
|---------------|---------|--------|
| Same location | 202     | 51     |
| 1 to 100      | 189     | 143    |
| 101 to 200    | 290     | 343    |
| 201 to 300    | 456     | 526    |

We can then attempt to replicate these results using the implementation presented here.

In [2]:
import pandas as pd
import numpy as np
import stats.knox as kx

Read the data and set up the bandwidths.

In [3]:
data = pd.read_csv("../data/chicago_burglary_2014_2017.csv", 
                   parse_dates=['date_single'], 
                   dayfirst=True)

data_subset = data[data['date_single'].between('2016-01-01', '2016-12-31')]
data_subset.to_csv("../data/chicago_burglary_2016.csv", 
                   columns=['x','y','date_single'], 
                   date_format='%d/%m/%Y', index=False)

xy = data_subset[['x','y']]
t = (data_subset['date_single'] - pd.to_datetime('2016-01-01')).dt.days

s_bands = 100 * np.arange(4)
s_bands[0] = 1
t_bands = 2*np.arange(1,3)

The test can now be run and the observed contingency table displayed:

In [4]:
result = kx.knox_test(xy, t, s_bands, t_bands, 9)
result.conting_observed

,"[0, 2]","(2, 4]"
"[0, 1]",225.0,50.0
"(1, 100]",260.0,142.0
"(100, 200]",457.0,341.0
"(200, 300]",715.0,567.0


Evidently this output does not match with that of the NRC: counts in the first column, for example, are substantially higher than for the NRC. This implies that there is a discrepancy in the calculation of the space-time contingency table between the two methods. Note that this is not due to any stochastic element within the Knox test - the computation of the observed table is purely deterministic.

## Explanation of the discrepancy

After extensive testing, the cause of the discrepancy between the two methods appears to relate to an (apparent) mis-match between the intervals reported by the NRC and those upon which its calculations are based.

In short, it appears that results that are labelled in the NRC as relating to an interval of **'a to b'** are in fact computed on the basis of the interval **'(a-1) to (b-1)'**. For example, a temporal band of **'3 to 4'** days in fact includes incidents that are either 2 or 3 days apart, and does not include incidents that are 4 days apart.

More specifically, it appears that what is labelled in the NRC as **'a to b'** in fact represents the interval **\[a-1, b)**. Since, in the temporal case, the NRC does not use time-of-day information (i.e. it uses the date only), 'strictly less than b' is equivalent to 'less than or equal to (b-1)' in practice.

The NRC results can be reproduced by the present implementation by adding the `interval_side='right'` flag when calling the Knox test. This determines on which sides the intervals are open/closed in the space-time bands (the default behaviour is 'left').

In [5]:
result = kx.knox_test(xy, t, s_bands, t_bands, 9, interval_side='right')
result.conting_observed

,"[0, 2)","[2, 4)"
"[0, 1)",202.0,51.0
"[1, 100)",189.0,143.0
"[100, 200)",290.0,343.0
"[200, 300)",456.0,526.0


...which matches the results given by the NRC, displayed in the previous section. Although the stochastic nature of the NRC makes it impossible to confirm with certainty, this issue appears to persist in both the 'statistical significance' and 'Knox ratio' tables produced by the NRC; that is, this appears to be the case for the calculator as a whole.

## Pathological examples

A minimal illustration of the issue can be produced by constructing a trivial pathological example. A simple dataset, containing only two incidents, can be constructed as follows: 

In [6]:
pathological_data = pd.DataFrame({'x': [0, 0],
                                  'y': [0, 100],
                                  't': pd.to_datetime(['2019/01/01', '2019/01/04'])})
pathological_data.to_csv("../data/pathological_example.csv", date_format='%d/%m/%Y', index=False)
pathological_data

,x,y,t
0,0,0,2019-01-01
1,0,100,2019-01-04


Trivially, there is one pair of events here, with spatial distance 100 metres and temporal distance 3 days. 

The NRC was run on this data, firstly with 2 spatial bands of width 200 metres and 2 temporal bands of width 3 days. The NRC gives the following output for the observed contingency table:

| &nbsp;        |  0 to 3 | 4 to 6 |
|---------------|---------|--------|
| Same location | 0     | 0    |
| 1 to 200      | 0     | 1    |
| 201 to 400    | 0     | 0    |

In this case, the single pair has been categorised as falling within the '4 to 6 days' category, although the events are only 3 days apart.

The equivalent issue can also be seen in the spatial dimension: running the same procedure but instead with a spatial bandwidth of 100 metres gives:

| &nbsp;        |  0 to 3 | 4 to 6 |
|---------------|---------|--------|
| Same location | 0     | 0    |
| 1 to 100      | 0     | 0    |
| 101 to 200    | 0     | 1    |

In this case, the spatial separation between the events has been categorised as '101 to 200 metres'.

## Implications

Conceptually, the issue with the NRC is not an issue of labelling, since the labels correctly reflect the parameters specified by the user; rather, the issue is with the underlying calculations. The results do not correspond to those requested by the user, in the sense that they are out by 1 unit.

In practical terms, this issue is much more significant in the temporal dimension than the spatial dimension: the scales involved mean that a difference of one metre (or one foot, *etc*) is of little significance in analysis of this type. Since many studies employ temporal bandwidths between 1 and 7 days, however, the issue represents a much more substantial discrepancy. In particular, since the largest (and most practically consequential) effects are typically found at the shortest scales, the most notable implications are in the initial bands.

The apparent issue means that results generated using the NRC which refer to an effect at '0 to 3 days', for example, actually reflect the effect at '0 to 2 days' (strictly this is 'less than 3 days', but since the NRC operates with date information only, these are equivalent). Since it is almost ubiquitous in data that clustering effects decay over time, this means that reported results for these initial temporal bands are likely to over-estimate the true values (with respect to both significance and effect size). Equivalently, when the NRC is used to estimate the extent of a clustering pattern (i.e. identify the period of time over which a significant pattern exists), this is also likely to be over-estimated.

Because the temporal decay is particularly pronounced at short timescales, the over-estimation described here is likely to be greatest when short temporal bands are used: the difference in effect between '0 to 2 days' and '0 to 1 day' is greater than that between '0 to 7 days' and '0 to 6 days', for example. In general, therefore, the risk and extent of over-estimation is likely to be highest in applications of the NRC that employ these particularly short bandwidths.